In [13]:
%cd "../"

/home/jovyan/JupyterLabRoot/aoa/AoaDemoModels/model_definitions/78036940-248e-4dae-b690-6f70eefcb188/DOCKER/model_modules


In [33]:
# %load ../training.py
import os

from teradataml import create_context
from teradataml.context.context import get_connection


def train(data_conf, model_conf, **kwargs):
    hyperparams = model_conf["hyperParameters"]

    create_context(host=data_conf["hostname"],
                   username=os.environ["TD_USERNAME"],
                   password=os.environ["TD_PASSWORD"])

    print("Starting training...")

    execute_sql_script(get_connection(), "training.sql")
    
    print("Finished training")

    # save model 
    # get_connection().execute("INSERT INTO {} SELECT {}, T.* FROM {} T"
    #                         .format(data_conf["model_table"], '1', "csi_telco_churn_model"))

    print("Saved trained model")


In [31]:
# %load ../util.py

def execute_sql_script(conn, filename):
    with open(filename, 'r') as f:
        cmds = f.read().split(';')
        
    for cmd in cmds:
        cmd = cmd.strip()
        if cmd:
            print("Executing cmd: {}".format(cmd))
            conn.execute(cmd)

In [19]:
import json
import os
import getpass

model_conf = json.load(open("../config.json", "r"))
data_conf = {
        "hostname": "tddb-env-d-268.vantage.demo.intellicloud.teradata.com",
        "data_table": "PIMA_TRAIN",
        "model_table": "aoa_models"
    }

os.environ["TD_USERNAME"] = input("Username: ")
os.environ["TD_PASSWORD"] = getpass.getpass("Password:")

Username:  wf250003
Password: ········


In [34]:
train(data_conf, model_conf)

/opt/conda/lib/python3.6/site-packages/teradataml/context/context.py:144: UserWarning: [Teradata][teradataml](TDML_2002) Overwriting an existing context associated with Teradata connection. Most of the operations on any teradataml DataFrames created before this will not work.
  warnings.warn(Messages.get_message(MessageCodes.OVERWRITE_CONTEXT))


Starting training...
Executing cmd: DROP TABLE csi_telco_churn_model
Executing cmd: CREATE MULTISET TABLE csi_telco_churn_model 
AS (
  SELECT * FROM NaiveBayesTextClassifierTrainer (
    ON (
      SELECT * FROM NaiveBayesTextClassifierInternal (
        ON ( SELECT *
        FROM model_dataset_train
        ) AS "input" PARTITION BY category
        USING
        TokenColumn ('token')
       	ModelType ('Bernoulli')
        DocIDColumns ('customerid')
        DocCategoryColumn ('category')
      ) AS alias_1
    ) PARTITION BY 1
  ) AS alias_2 
)
WITH DATA
Finished training
Saved trained model


In [36]:
# %load scoring.py
import os
import json

from teradataml import create_context
from teradataml.dataframe.dataframe import DataFrame
from teradataml.context.context import get_connection


def evaluate(data_conf, model_conf, **kwargs):
    create_context(host=data_conf["hostname"],
                   username=os.environ["TD_USERNAME"],
                   password=os.environ["TD_PASSWORD"])
    
    print("Starting evaluation...")

    execute_sql_script(get_connection(), "scoring.sql")
    
    print("Finished evaluation")
    
    stats = DataFrame("telco_stat_output").to_pandas()
    metrics = dict(zip(stats.key, stats.value))
    
    with open("models/evaluation.json", "w+") as f:
        json.dump(metrics, f)

    

In [41]:
data_conf = {
        "hostname": "tddb-env-d-268.vantage.demo.intellicloud.teradata.com",
        "data_table": "PIMA_TEST",
        "model_table": "aoa_models"
    }
evaluate(data_conf, model_conf)

/opt/conda/lib/python3.6/site-packages/teradataml/context/context.py:144: UserWarning: [Teradata][teradataml](TDML_2002) Overwriting an existing context associated with Teradata connection. Most of the operations on any teradataml DataFrames created before this will not work.
  warnings.warn(Messages.get_message(MessageCodes.OVERWRITE_CONTEXT))


Starting evaluation...
Executing cmd: DROP TABLE csi_telco_churn_predict_test
Executing cmd: DROP TABLE telco_count_output
Executing cmd: DROP TABLE telco_stat_output
Executing cmd: DROP TABLE telco_acc_output
Executing cmd: CREATE TABLE csi_telco_churn_predict_test
AS
(SELECT *
    FROM NaiveBayesTextClassifierPredict@coprocessor (
    ON (SELECT *
        FROM model_dataset_test
    ) AS predicts 
    PARTITION BY customerid
    ON csi_telco_churn_model AS model DIMENSION
    USING
    InputTokenColumn ('token')
    ModelType ('Bernoulli')
    DocIDColumns ('customerid')
    TopK (1)
  ) AS dt
)
WITH DATA
Executing cmd: REPLACE VIEW csi_telco_churn_results AS 
    SELECT P.customerid, P.prediction, P.loglik, T.category actual
        FROM csi_telco_churn_predict_test P 
        JOIN model_dataset_test T ON P.customerid = T.customerid
Executing cmd: SELECT * FROM ConfusionMatrix (
  ON csi_telco_churn_results
    PARTITION BY 1
    OUT TABLE CountTable (telco_count_output)
    OUT TAB

In [42]:
with open("models/evaluation.json") as f:
    print(json.loads(f.read()))

{'Mcnemar Test P-Value': '0', 'Null Error Rate': '0.0003', 'Kappa': '0.9964', 'Accuracy': '0.9999', 'P-Value [Acc > NIR]': '0', '95% CI': '(0.9999, 0.9999)'}
